In [25]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import metrics
from read_tfrecord import *
from mtcnn_model import Pnet,cls_ohem,bbox_ohem
from tqdm import tqdm
import os
import numpy as np

In [18]:
data_path = "data/12/train_PNet_landmark.tfrecord_shuffle"

In [19]:
def load_pokemon(mode='train'):
    """ 加载pokemon数据集的工具！
    :param root:    数据集存储的目录
    :param mode:    mode:当前加载的数据是train,val,还是test
    :return:
    """
    # # 创建数字编码表,范围0-4;
    # name2label = {}  # "sq...":0   类别名:类标签;  字典 可以看一下目录,一共有5个文件夹,5个类别：0-4范围;
    # for name in sorted(os.listdir(os.path.join(root))):     # 列出所有目录;
    #     if not os.path.isdir(os.path.join(root, name)):
    #         continue
    #     # 给每个类别编码一个数字
    #     name2label[name] = len(name2label.keys())
 
    # 读取Label信息;保存索引文件images.csv
    # [file1,file2,], 对应的标签[3,1] 2个一一对应的list对象。
    # 根据目录,把每个照片的路径提取出来,以及每个照片路径所对应的类别都存储起来，存储到CSV文件中。
    size = 12
    images,labels,boxes,landmarks = red_tf(data_path,size)
 
    # 图片切割成，训练70%，验证15%，测试15%。
    if mode == 'train':                                                     # 70% 训练集
        images = images[:int(0.7 * len(images))]
        labels = labels[:int(0.7 * len(labels))]
        boxes  = boxes[:int(0.7 * len(boxes))]
        landmarks = landmarks[:int(0.7 * len(landmarks))]
    elif mode == 'val':                                                     # 15% = 70%->85%  验证集
        images = images[int(0.7 * len(images)):int(0.85 * len(images))]
        labels = labels[int(0.7 * len(labels)):int(0.85 * len(labels))]
        boxes = boxes[int(0.7 * len(boxes)):int(0.85 * len(boxes))]
        landmarks = landmarks[int(0.7 * len(landmarks)):int(0.85 * len(landmarks))]
    else:                                                                   # 15% = 70%->85%  测试集
        images = images[int(0.85 * len(images)):]
        labels = labels[int(0.85 * len(labels)):]
        boxes = boxes[int(0.85 * len(boxes)):]
        landmarks = landmarks[int(0.85 * len(landmarks)):]
    ima = tf.data.Dataset.from_tensor_slices(images)
    lab = tf.data.Dataset.from_tensor_slices(labels)
    roi = tf.data.Dataset.from_tensor_slices(boxes)
    land = tf.data.Dataset.from_tensor_slices(landmarks)
    # ima,lab,roi = preprocess(ima,lab,roi)
    train_data = tf.data.Dataset.zip((ima, lab, roi,land)).shuffle(1000).batch(32)
    train_data = train_data.as_numpy_iterator()
    return train_data

In [20]:
for i,(img,lab,boxes,landmarks) in enumerate(load_pokemon("train")):
    print(i)
    print(lab)
    print(boxes)
    print(landmarks)

<MapDataset shapes: {image/encoded: (), image/label: (), image/landmark: (10,), image/roi: (4,)}, types: {image/encoded: tf.string, image/label: tf.int64, image/landmark: tf.float32, image/roi: tf.float32}>
0
[ 0. -1. -2.  0.  1.  0.  0.  0.  1.  0.  0.  0.  1.  1.  0.  0.  0. -1.
  0.  0.  0.  0. -1.  0.  1. -1.  0.  0. -1.  0.  1.  0.]
[[ 0.    0.    0.    0.  ]
 [ 0.17  0.16 -0.14  0.02]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [-0.07 -0.05  0.04  0.24]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.03  0.08 -0.22 -0.05]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.03 -0.    0.2   0.12]
 [ 0.09 -0.04 -0.1  -0.09]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.24  0.17  0.   -0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.19  0.06 -0.27 -0.14]
 [ 0.    0.    0.    0.  ]
 [-0.01 -0.0

  0.         0.         0.         0.        ]]
52
[ 0. -1.  1.  0.  0. -1. -2.  0.  1.  0.  0.  0.  1.  0. -2. -1.  0. -1.
  1. -1.  0.  0. -2. -1.  0.  0.  1. -1.  0.  1.  0.  0.]
[[ 0.    0.    0.    0.  ]
 [-0.04 -0.21 -0.1   0.19]
 [ 0.   -0.1  -0.02  0.11]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [-0.17 -0.42 -0.02  0.1 ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [-0.03  0.   -0.21 -0.14]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.11 -0.16 -0.    0.21]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.14  0.09 -0.25 -0.08]
 [ 0.    0.    0.    0.  ]
 [ 0.09  0.14 -0.24  0.  ]
 [-0.09 -0.06 -0.02  0.24]
 [-0.26 -0.27 -0.05  0.48]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.22  0.11  0.07  0.05]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.22 -0.06  0.08  0.04]
 [ 0.09 -0.15  0.09  0.44]
 [ 0.    0.    0.    0.  ]
 [-0.18 -0.01 -0.06  0.11]
 [ 0.   

 [ 0.08  0.04 -0.02  0.14]]
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.     

  0.         0.         0.         0.        ]]
148
[-1.  0.  0.  0.  1.  0. -2.  0.  0.  0. -1.  0.  0.  0.  0. -1.  0. -1.
 -2.  0.  0.  0.  0.  0.  0. -1.  0. -1. -1.  0. -1. -1.]
[[ 0.16  0.03 -0.17 -0.16]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [-0.01  0.06 -0.07  0.13]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.08 -0.02 -0.28 -0.2 ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.13  0.1  -0.26  0.  ]
 [ 0.    0.    0.    0.  ]
 [-0.18 -0.22 -0.1  -0.04]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [-0.27  0.07 -0.06  0.32]
 [ 0.    0.    0.    0.  ]
 [ 0.28  0.04 -0.08 -0.07]
 [-0.01 -0.08 -0.16 -0.2 ]
 [ 0.    0.    0.    0.  ]
 [ 0.26

  0.         0.         0.         0.        ]]
196
[-1.  0.  0.  0.  0.  0. -1. -1. -2.  0.  0.  0.  0.  1.  1.  1.  0.  0.
  0.  0.  0. -1.  0. -1.  0.  0. -1.  0. -1.  1.  0.  0.]
[[ 0.22  0.27 -0.16  0.05]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [-0.16 -0.28 -0.04  0.83]
 [ 0.11  0.08  0.23  0.52]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.03  0.18 -0.06  0.12]
 [-0.05 -0.1   0.11  0.22]
 [ 0.2   0.09 -0.03 -0.03]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.19  0.16 -0.11 -0.02]
 [ 0.    0.    0.    0.  ]
 [ 0.35 -0.25 -0.08 -0.03]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.24  0.18 -0.05  0.01]
 [ 0.    0.    0.    0.  ]
 [ 0.1   0.04 -0.13 -0.15]
 [ 0.   -0.03 -0.03  0.13]
 [ 0.  

...

 [ 0.31 -0.06 -0.03 -0.21]]
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.46495327 0.36682242 0.84345794 0.46962616 0.7266355  0.61915886
  0.43691587 0.83878505 0.7313084  0.9135514 ]
 [0.25       0.12654321 0.7191358  0.09567901 0.55246913 0.4722222
  0.34876543 0.75617284 0.68209875 0.7191358 ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.    

  0.         0.         0.         0.        ]]
343
[ 1.  0. -1.  1.  1.  0.  1.  1.  0.  0.  0.  1.  0.  1.  0.  0. -1.  1.
  0. -1.  1. -2.  0.  0.  0. -2.  0.  0.  0.  1. -1. -1.]
[[-0.03  0.07 -0.15  0.16]
 [ 0.    0.    0.    0.  ]
 [ 0.2   0.2   0.01  0.04]
 [-0.07  0.01 -0.15 -0.12]
 [ 0.03 -0.24 -0.09 -0.05]
 [ 0.    0.    0.    0.  ]
 [-0.07 -0.29 -0.04 -0.07]
 [ 0.03  0.04 -0.22 -0.06]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.   -0.15 -0.19  0.05]
 [ 0.    0.    0.    0.  ]
 [-0.01 -0.09  0.14  0.03]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [-0.14  0.06  0.04  0.44]
 [-0.08  0.01  0.03  0.06]
 [ 0.    0.    0.    0.  ]
 [ 0.28  0.05 -0.21 -0.1 ]
 [-0.04 -0.11 -0.14 -0.03]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [-0.04 -0.03 -0.14 -0.12]
 [ 0.26

  0.         0.         0.         0.        ]]
390
[-2.  1.  0.  0.  0.  0.  0.  0.  0.  0. -2.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0. -1.  0. -2.  1.  0.  0.  0.  1.  0.  0.  0.]
[[ 0.    0.    0.    0.  ]
 [ 0.02  0.01 -0.14 -0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [-0.03  0.03 -0.17  0.18]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [-0.13 -0.15 -0.12  0.17]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [-0.06 -0.29 -0.05 -0.02]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [-0.07  0.06 -0.02  0.16]
 [ 0.    0.    0.    0.  ]
 [ 0.  

435
[ 0.  0.  0.  0.  1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0. -2. -1.  0.
  1.  0.  0. -1. -1.  0.  1.  0.  1.  0.  0.  0.  0.  0.]
[[ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.06  0.05 -0.1  -0.04]
 [-0.05 -0.19  0.16  0.23]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.21 -0.26 -0.23  0.05]
 [ 0.    0.    0.    0.  ]
 [-0.11 -0.14 -0.11 -0.1 ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.1   0.22 -0.17 -0.  ]
 [ 0.17  0.07 -0.13  0.07]
 [ 0.    0.    0.    0.  ]
 [ 0.21 -0.06 -0.07 -0.03]
 [ 0.    0.    0.    0.  ]
 [ 0.05  0.03 -0.17  0.16]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]]


  0.         0.         0.         0.        ]]
486
[ 0. -1.  1. -1. -1.  1.  1.  0. -1.  0.  1.  0.  0.  1.  0. -1.  0.  0.
  0.  1.  0. -2.  0.  0. -1. -2.  1.  0. -2. -1.  0.  0.]
[[ 0.    0.    0.    0.  ]
 [ 0.04  0.12 -0.25  0.04]
 [ 0.12  0.02 -0.03  0.03]
 [-0.24 -0.57 -0.04  0.44]
 [ 0.    0.07 -0.2   0.33]
 [-0.06 -0.06 -0.21 -0.03]
 [-0.03 -0.    0.1   0.28]
 [ 0.    0.    0.    0.  ]
 [ 0.19  0.11 -0.09  0.19]
 [ 0.    0.    0.    0.  ]
 [-0.09 -0.17 -0.08  0.13]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [-0.09 -0.05 -0.07  0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.28 -0.21 -0.02 -0.15]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.11 -0.07  0.1 ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.08 -0.02  0.31  0.64]
 [ 0.    0.    0.    0.  ]
 [-0.04 -0.09  0.02  0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.28 -0.02 -0.   -0.16]
 [ 0.  